In [13]:
import pandas as pd
import pickle

In [14]:
model_path = "trained_model.pkl"
with open(model_path, "rb") as file:
    model = pickle.load(file)

In [15]:
data_path = "Troop.parquet.gzip"
df_large = pd.read_parquet(data_path)

In [16]:
df_large.head()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako


In [17]:
df_large_encoded = pd.get_dummies(df_large, columns=["unit_type", "homeworld"])

In [18]:
missing_cols = set(model.feature_names_in_) - set(df_large_encoded.columns)
for col in missing_cols:
    df_large_encoded[col] = 0

In [19]:
df_large_encoded = df_large_encoded[model.feature_names_in_]
df_large_encoded.head()

,homeworld_Alderaan,homeworld_Aleen Minor,homeworld_Bestine IV,homeworld_Cerea,homeworld_Champala,homeworld_Chandrila,homeworld_Concord Dawn,homeworld_Corellia,homeworld_Dagobah,homeworld_Dathomir,...,homeworld_Vulpter,homeworld_Zolan,unit_type_at-at,unit_type_at-st,unit_type_resistance_soldier,unit_type_stormtrooper,unit_type_tie_fighter,unit_type_tie_silencer,unit_type_unknown,unit_type_x-wing
0,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,0,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,0,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,0,False


In [20]:
df_large ["predictions"] = model.predict(df_large_encoded)
df_large.head()

,timestamp,unit_id,unit_type,location_x,location_y,destination_x,destination_y,homeworld,predictions
0,2023-06-03 03:19:15,919214,tie_silencer,2.0,5.0,9,3,Aleen Minor,empire
1,2023-02-24 13:50:40,9467154,stormtrooper,9.0,0.0,9,1,Malastare,empire
2,2023-03-29 19:54:55,6585778,tie_silencer,0.0,6.0,5,9,Serenno,empire
3,2023-04-30 00:58:11,3878023,tie_silencer,4.0,2.0,9,9,Tund,empire
4,2023-04-10 22:00:26,5537117,at-st,6.0,8.0,5,8,Skako,empire


In [21]:
output_path = "predicted_troop_movements.parquet"
df_large.to_parquet(output_path, compression="gzip")

print(f"Predictions saved to {output_path}")

Predictions saved to predicted_troop_movements.parquet
